In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import json

이름 basicList_link__JLQJf<br>
특징 basicList_desc__3kwkT basicList_max__ODs_o

In [35]:
# 페이지 바꿔가며 크롤링 
goods_name = []
feature_list = []
for i in range(1,6) :
    url = requests.get(f'https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=%EC%A0%84%EB%8F%99%ED%82%A5%EB%B3%B4%EB%93%9C&pagingIndex={i}&pagingSize=20&productSet=total&query=%EC%A0%84%EB%8F%99%ED%82%A5%EB%B3%B4%EB%93%9C&sort=rel&timestamp=&viewType=list')
    web_html = BeautifulSoup(url.text, 'html.parser')
    goods = web_html.find_all(class_='basicList_link__JLQJf')
    for i in range(len(goods)-1) :
        goods_name.append(goods[i].get_text())

    features = web_html.find_all(class_='basicList_detail_box__OoXKt')
    for i in range(len(features)) :
        feature_list.append(features[i].get_text())

In [49]:
' '.join(goods_name[0].split(' ')[1:])

'레드윙 써클 접이식 전기 전동킥보드 10인치 36V 10Ah'

In [50]:
# 상품목록에서 제조사,상품명 추출 
company = []
goods_list = []
for goods in goods_name :
    company.append(goods.split(' ')[0])
    goods_list.append(' '.join(goods.split(' ')[1:]))


In [37]:
# feature_list에서 충전시간, 주행거리만 추출

charge_time = []
drive_distance = []

for i in range(len(feature_list)) :
    충전시간 = feature_list[i].find('충전시간')+7
    for j in range(충전시간, len(feature_list[i])) :
        if feature_list[i][j].isdigit() or feature_list[i][j] == '.' :
            pass
        else :
            charge_time.append(feature_list[i][충전시간:j])
            break

for i in range(len(feature_list)) :
    주행거리 = feature_list[i].find('주행거리')+7
    for j in range(주행거리, len(feature_list[i])) :
        if feature_list[i][j].isdigit() or feature_list[i][j] == '.' :
            pass
        else :
            drive_distance.append(feature_list[i][주행거리:j])
            break


In [53]:
df = pd.DataFrame([company,goods_list,charge_time,drive_distance]).T
df.columns=['제조사','상품명','충전시간(시간)','주행거리(km)']

transform = df.to_json(force_ascii=False, orient = 'index', indent= 4)
transform_2 = json.loads(transform)

In [57]:
# file 저장 
with open('./output.json','w',encoding='utf-8') as json_file :
    json.dump(transform_2,json_file,ensure_ascii=False)